In [3]:
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
import keras as k

Using TensorFlow backend.


In [4]:
market_train_df = pickle.load(open('Market_train',"rb"))
news_train_df = pickle.load(open('News_train', 'rb'))

In [5]:
market_train_df.head()

,time,assetCode,assetName,volume,close,open,returnsClosePrevRaw1,returnsOpenPrevRaw1,returnsClosePrevMktres1,returnsOpenPrevMktres1,returnsClosePrevRaw10,returnsOpenPrevRaw10,returnsClosePrevMktres10,returnsOpenPrevMktres10,returnsOpenNextMktres10,universe
0,2007-02-01 22:00:00+00:00,A.N,Agilent Technologies Inc,2606900.0,32.19,32.17,0.005938,0.005312,NaN,NaN,-0.001860,0.000622,NaN,NaN,0.034672,1.0
1,2007-02-01 22:00:00+00:00,AAI.N,AirTran Holdings Inc,2051600.0,11.12,11.08,0.004517,-0.007168,NaN,NaN,-0.078708,-0.088066,NaN,NaN,0.027803,0.0
2,2007-02-01 22:00:00+00:00,AAP.N,Advance Auto Parts Inc,1164800.0,37.51,37.99,-0.011594,0.025648,NaN,NaN,0.014332,0.045405,NaN,NaN,0.024433,1.0
3,2007-02-01 22:00:00+00:00,AAPL.O,Apple Inc,23747329.0,84.74,86.23,-0.011548,0.016324,NaN,NaN,-0.048613,-0.037182,NaN,NaN,-0.007425,1.0
4,2007-02-01 22:00:00+00:00,ABB.N,ABB Ltd,1208600.0,18.02,18.01,0.011791,0.025043,NaN,NaN,0.012929,0.020397,NaN,NaN,-0.017994,1.0


In [6]:
news_train_df.head()

,time,sourceTimestamp,firstCreated,sourceId,headline,urgency,takeSequence,provider,subjects,audiences,...,noveltyCount12H,noveltyCount24H,noveltyCount3D,noveltyCount5D,noveltyCount7D,volumeCounts12H,volumeCounts24H,volumeCounts3D,volumeCounts5D,volumeCounts7D
0,2007-01-01 04:29:32+00:00,2007-01-01 04:29:32+00:00,2007-01-01 04:29:32+00:00,e58c6279551b85cf,China's Daqing pumps 43.41 mln tonnes of oil i...,3,1,RTRS,"{'ENR', 'ASIA', 'CN', 'NGS', 'EMRG', 'RTRS', '...","{'Z', 'O', 'OIL'}",...,0,0,0,0,0,0,0,3,6,7
1,2007-01-01 07:03:35+00:00,2007-01-01 07:03:34+00:00,2007-01-01 07:03:34+00:00,5a31c4327427f63f,"FEATURE-In kidnapping, finesse works best",3,1,RTRS,"{'FEA', 'CA', 'LATAM', 'MX', 'INS', 'ASIA', 'I...","{'PGE', 'PCO', 'G', 'ESN', 'MD', 'PCU', 'DNP',...",...,1,1,1,1,1,1,1,3,3,3
2,2007-01-01 11:29:56+00:00,2007-01-01 11:29:56+00:00,2007-01-01 11:29:56+00:00,1cefd27a40fabdfe,PRESS DIGEST - Wall Street Journal - Jan 1,3,1,RTRS,"{'RET', 'ENR', 'ID', 'BG', 'US', 'PRESS', 'IQ'...","{'T', 'DNP', 'PSC', 'U', 'D', 'M', 'RNP', 'PTD...",...,0,0,0,0,0,0,0,5,11,17
3,2007-01-01 12:08:37+00:00,2007-01-01 12:08:37+00:00,2007-01-01 12:08:37+00:00,23768af19dc69992,PRESS DIGEST - New York Times - Jan 1,3,1,RTRS,"{'FUND', 'FIN', 'CA', 'SFWR', 'INS', 'PUB', 'B...","{'T', 'DNP', 'PSC', 'U', 'D', 'M', 'RNP', 'PTD...",...,0,0,0,0,0,0,0,5,13,15
4,2007-01-01 12:08:37+00:00,2007-01-01 12:08:37+00:00,2007-01-01 12:08:37+00:00,23768af19dc69992,PRESS DIGEST - New York Times - Jan 1,3,1,RTRS,"{'FUND', 'FIN', 'CA', 'SFWR', 'INS', 'PUB', 'B...","{'T', 'DNP', 'PSC', 'U', 'D', 'M', 'RNP', 'PTD...",...,0,0,0,0,0,0,0,0,0,0


In [7]:
def preprocess_data(mkt_train_df, news_train_df):
    mkt_train_df['time'] = mkt_train_df['time'].dt.date
    news_train_df['time'] = news_train_df['time'].dt.date
    assetCodes = []
    index = 0
    for x in news_train_df['assetCodes']:
        x = x.split(',')[0].split("'")[1]
        assetCodes.append(x)
    news_train_df['assetCode'] = np.asarray(assetCodes)
    irrelevantColumns = ['sourceTimestamp', 'firstCreated', 'sourceId', 
                         'headline', 'provider', 'subjects', 'audiences',
                        'headlineTag', 'marketCommentary', 'assetCodes', 'assetName']
    news_train_df.drop(irrelevantColumns, axis=1, inplace=True)
    market_train_df.drop(['assetName'], axis=1, inplace=True)
    modifiednews = news_train_df.groupby(['time','assetCode'], sort=False).aggregate(np.mean).reset_index()
    
    # join news reports to market data, note many assets will have many days without news data
    merged = pd.merge(mkt_train_df, modifiednews, how='left', on=['time', 'assetCode'], copy=False) 
    merged = merged.fillna(0)
    return merged

In [8]:
def normalizeY(ydf):
    ydf = (ydf + 1) / 2
    return ydf


In [9]:
X = preprocess_data(market_train_df, news_train_df)
y = X['returnsOpenNextMktres10']
X.drop(['returnsOpenNextMktres10'], axis=1, inplace=True)
y = normalizeY(y)


In [10]:
X_new = X[['assetCode','close','open','sentimentClass','sentimentNegative','sentimentNeutral','sentimentPositive']]
X_new = pd.concat((X_new,y), axis = 1)
X_new.fillna(0)
print(X_new.shape)

(4072956, 8)


In [11]:
#Get labels
labels = X_new.assetCode
labels = pd.get_dummies(labels)



In [12]:
#Convert label to argmax
label_n = labels.values


In [13]:
#Convert to integer values
label_n = np.argmax(label_n, axis= 1)

In [14]:
#Convert to DF
label_pd = pd.DataFrame(label_n)

In [15]:
X_temp = X_new.drop(labels=['assetCode'], axis=1)

In [16]:
X_final = pd.concat((label_pd,X_temp), axis=1)

In [17]:

df_stock = X_final.copy()  #Get values for stock 

In [66]:
df_stock.shape

(4072956, 8)

In [67]:
a = df_stock[df_stock.returnsOpenNextMktres10<1]
a = df_stock[df_stock.returnsOpenNextMktres10>0]
print(a.shape)

(4072819, 8)


In [70]:
a.head()

,0,close,open,sentimentClass,sentimentNegative,sentimentNeutral,sentimentPositive,returnsOpenNextMktres10
0,0,32.19,32.17,0.50,0.080418,0.568264,0.351318,0.517336
1,2,11.12,11.08,0.25,0.138128,0.412758,0.449114,0.513902
2,7,37.51,37.99,0.00,0.000000,0.000000,0.000000,0.512216
3,8,84.74,86.23,0.00,0.000000,0.000000,0.000000,0.496288
4,15,18.02,18.01,0.00,0.000000,0.000000,0.000000,0.491003


In [100]:
X = a.iloc[:,0:-1]
Y = a.iloc[:,-1]
print(X.shape)

(4072819, 7)


In [90]:
def build_model(lre):
    model = k.Sequential()
    model.add(k.layers.Dense(units=256, activation='relu', input_dim= 7))
    model.add(k.layers.Dense(units=512, activation='relu'))
    model.add(k.layers.Dense(units=256, activation='relu'))
    model.add(k.layers.Dense(units=64,activation='sigmoid'))
    model.add(k.layers.Dense(units=1, activation='sigmoid'))
    adam_o = k.optimizers.Adamax(lr=lre)
    model.compile(optimizer = adam_o, loss = 'mean_squared_error',metrics = ['accuracy','loss'])
    model.summary()
    return model
    

In [91]:
model = build_model(0.001)
tensorboard = 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_36 (Dense)             (None, 200)               1600      
_________________________________________________________________
dense_37 (Dense)             (None, 200)               40200     
_________________________________________________________________
dense_38 (Dense)             (None, 200)               40200     
_________________________________________________________________
dense_39 (Dense)             (None, 30)                6030      
_________________________________________________________________
dense_40 (Dense)             (None, 1)                 31        
Total params: 88,061
Trainable params: 88,061
Non-trainable params: 0
_________________________________________________________________


In [92]:
history  = model.fit(X,Y,batch_size=8000,epochs=10, validation_split=0.2)

Train on 3258255 samples, validate on 814564 samples
Epoch 1/10
3258255/3258255 [==============================] - 27s 8us/step - loss: 15.5935 - val_loss: 0.0016
Epoch 2/10
3258255/3258255 [==============================] - 26s 8us/step - loss: 15.5930 - val_loss: 0.0014
Epoch 3/10
3258255/3258255 [==============================] - 26s 8us/step - loss: 15.5930 - val_loss: 0.0014
Epoch 4/10
3258255/3258255 [==============================] - 26s 8us/step - loss: 15.5930 - val_loss: 0.0014
Epoch 5/10
3258255/3258255 [==============================] - 26s 8us/step - loss: 15.5930 - val_loss: 0.0014
Epoch 6/10
3258255/3258255 [==============================] - 26s 8us/step - loss: 15.5930 - val_loss: 0.0014
Epoch 7/10
3258255/3258255 [==============================] - 26s 8us/step - loss: 15.5930 - val_loss: 0.0014
Epoch 8/10
3258255/3258255 [==============================] - 26s 8us/step - loss: 15.5930 - val_loss: 0.0020
Epoch 9/10
3258255/3258255 [==============================] - 26s 8

In [115]:
b = X.iloc[59,0:7].values
b = np.reshape(b,[1,7])
print(b.shape)

(1, 7)


In [116]:
model.predict(b)

array([[0.5170137]], dtype=float32)

In [117]:
Y.iloc[1]

0.5139016398041771